In [1]:

from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [65]:
src = (ImageList.from_folder(path='train').split_by_rand_pct(0.2).label_from_folder())

In [66]:
data = (src.transform(([],[]), size=48, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=64).normalize())

In [67]:
data.train_ds.y.items

array([ 0,  0,  0,  0, ..., 44, 44, 44, 44], dtype=int64)

In [68]:
len(data.train_ds.y.items)

65934

In [69]:
x,y = np.unique(data.train_ds.y.items,return_counts=True)

In [70]:
dict(zip(data.classes,y))

{'class-1059': 724,
 'class-128': 603,
 'class-140': 1197,
 'class-1625': 1321,
 'class-204': 8830,
 'class-239': 537,
 'class-273': 456,
 'class-326': 515,
 'class-337': 2763,
 'class-362': 1749,
 'class-390': 1819,
 'class-394': 971,
 'class-4': 1192,
 'class-448': 2025,
 'class-450': 815,
 'class-460': 1113,
 'class-508': 1656,
 'class-526': 501,
 'class-536': 1093,
 'class-540': 832,
 'class-543': 1097,
 'class-561': 416,
 'class-581': 1519,
 'class-597': 793,
 'class-629': 428,
 'class-635': 953,
 'class-639': 1779,
 'class-653': 589,
 'class-654': 866,
 'class-663': 773,
 'class-67': 2593,
 'class-697': 3667,
 'class-707': 909,
 'class-72': 702,
 'class-734': 473,
 'class-771': 4377,
 'class-78': 1713,
 'class-784': 423,
 'class-804': 2013,
 'class-811': 735,
 'class-854': 1416,
 'class-857': 898,
 'class-872': 4442,
 'class-957': 483,
 'class-966': 1165}

In [20]:
df_sub = pd.read_csv('submission-7.csv',low_memory=False
        )

In [21]:
df_sub.head()

,filename,agkistrodon_contortrix,agkistrodon_piscivorus,boa_imperator,carphophis_amoenus,charina_bottae,coluber_constrictor,crotalus_adamanteus,crotalus_atrox,crotalus_horridus,...,regina_septemvittata,rhinocheilus_lecontei,storeria_dekayi,storeria_occipitomaculata,thamnophis_elegans,thamnophis_marcianus,thamnophis_ordinoides,thamnophis_proximus,thamnophis_radix,thamnophis_sirtalis
0,e5cf4ff0a9326e1eae50b7b4c6ddff19.jpg,0.001101,0.001290,0.000592,0.012547,0.000688,0.002600,0.000500,0.001253,0.000725,...,0.001283,0.000663,0.002120,0.001215,0.000632,0.000941,0.001305,0.000713,0.000452,0.001553
1,04c26281edbd155dca879084cdf860dc.jpg,0.012227,0.009883,0.006562,0.004459,0.004420,0.004619,0.005584,0.022746,0.023403,...,0.003802,0.009957,0.012157,0.004046,0.003008,0.006691,0.005566,0.006390,0.006932,0.648545
2,703e09e07ef754f4e60b7dfb3cad53bc.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,6930489ef4a47b44e717c351b1893019.jpg,0.051632,0.010804,0.010608,0.004762,0.024319,0.010939,0.009692,0.138383,0.019260,...,0.006025,0.016214,0.016145,0.007686,0.007105,0.008815,0.030455,0.006049,0.010626,0.062437
4,790d4ba419acd10b3f27c40abf641686.jpg,0.005229,0.007836,0.010057,0.003692,0.008069,0.008216,0.006193,0.018113,0.004133,...,0.004513,0.004308,0.108470,0.018880,0.209652,0.020386,0.018643,0.005512,0.011422,0.165548


In [24]:
df_new = df_sub.iloc[:,1:46]

In [25]:
df_new.head()

,agkistrodon_contortrix,agkistrodon_piscivorus,boa_imperator,carphophis_amoenus,charina_bottae,coluber_constrictor,crotalus_adamanteus,crotalus_atrox,crotalus_horridus,crotalus_pyrrhus,...,regina_septemvittata,rhinocheilus_lecontei,storeria_dekayi,storeria_occipitomaculata,thamnophis_elegans,thamnophis_marcianus,thamnophis_ordinoides,thamnophis_proximus,thamnophis_radix,thamnophis_sirtalis
0,0.001101,0.001290,0.000592,0.012547,0.000688,0.002600,0.000500,0.001253,0.000725,0.000762,...,0.001283,0.000663,0.002120,0.001215,0.000632,0.000941,0.001305,0.000713,0.000452,0.001553
1,0.012227,0.009883,0.006562,0.004459,0.004420,0.004619,0.005584,0.022746,0.023403,0.008770,...,0.003802,0.009957,0.012157,0.004046,0.003008,0.006691,0.005566,0.006390,0.006932,0.648545
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.051632,0.010804,0.010608,0.004762,0.024319,0.010939,0.009692,0.138383,0.019260,0.081868,...,0.006025,0.016214,0.016145,0.007686,0.007105,0.008815,0.030455,0.006049,0.010626,0.062437
4,0.005229,0.007836,0.010057,0.003692,0.008069,0.008216,0.006193,0.018113,0.004133,0.009408,...,0.004513,0.004308,0.108470,0.018880,0.209652,0.020386,0.018643,0.005512,0.011422,0.165548


In [42]:
temp=df_new.idxmax(axis=1)

In [32]:
df_class = pd.read_csv('class.csv')

In [33]:
df_class.head()

,original_class,class_idx
0,thamnophis_proximus,4
1,nerodia_sipedon,67
2,opheodrys_vernalis,72
3,crotalus_horridus,78
4,crotalus_pyrrhus,128


In [39]:
data_dict = df_class.set_index('original_class')['class_idx'].to_dict()

In [40]:
data_dict

{'thamnophis_proximus': 4,
 'nerodia_sipedon': 67,
 'opheodrys_vernalis': 72,
 'crotalus_horridus': 78,
 'crotalus_pyrrhus': 128,
 'nerodia_rhombifer': 140,
 'thamnophis_sirtalis': 204,
 'natrix_natrix': 239,
 'crotalus_adamanteus': 273,
 'charina_bottae': 326,
 'pituophis_catenifer': 337,
 'lampropeltis_triangulum': 362,
 'nerodia_erythrogaster': 390,
 'thamnophis_marcianus': 394,
 'lampropeltis_californiae': 448,
 'crotalus_ruber': 450,
 'rhinocheilus_lecontei': 460,
 'opheodrys_aestivus': 508,
 'thamnophis_ordinoides': 526,
 'thamnophis_radix': 536,
 'masticophis_flagellum': 540,
 'pantherophis_vulpinus': 543,
 'hierophis_viridiflavus': 561,
 'heterodon_platirhinos': 581,
 'pantherophis_emoryi': 597,
 'regina_septemvittata': 629,
 'haldea_striatula': 635,
 'diadophis_punctatus': 639,
 'nerodia_fasciata': 653,
 'storeria_occipitomaculata': 654,
 'crotalus_scutulatus': 663,
 'storeria_dekayi': 697,
 'crotalus_viridis': 707,
 'boa_imperator': 734,
 'pantherophis_obsoletus': 771,
 'lich

In [45]:
count=0;
for i in temp:
    temp[count]="class-"+str(data_dict[i])
    count += 1

In [46]:
temp

0         class-635
1         class-204
2         class-140
3         class-872
4         class-811
5         class-654
6         class-204
7         class-707
8         class-337
9         class-771
10         class-78
11        class-536
12        class-128
13        class-204
14        class-204
15        class-697
16        class-697
17        class-561
18        class-204
19        class-204
20        class-450
21        class-390
22        class-857
23        class-204
24        class-204
25        class-337
26        class-663
27        class-872
28        class-337
29        class-204
            ...    
17701     class-540
17702     class-771
17703     class-448
17704     class-771
17705      class-78
17706     class-771
17707     class-957
17708     class-697
17709     class-771
17710     class-204
17711     class-460
17712     class-508
17713     class-854
17714     class-697
17715    class-1625
17716     class-337
17717     class-639
17718     class-653
17719    class-1625


In [73]:
dict(zip(data.classes,y))

{'class-1059': 0.010980677647344315,
 'class-128': 0.009145509145509146,
 'class-140': 0.018154518154518153,
 'class-1625': 0.020035186701853367,
 'class-204': 0.13392180058846725,
 'class-239': 0.008144508144508145,
 'class-273': 0.006916006916006916,
 'class-326': 0.007810841144174478,
 'class-337': 0.0419055419055419,
 'class-362': 0.026526526526526525,
 'class-390': 0.02758819425486092,
 'class-394': 0.014726848060181394,
 'class-4': 0.018078684745351414,
 'class-448': 0.030712530712530713,
 'class-450': 0.012360845694179027,
 'class-460': 0.01688051688051688,
 'class-508': 0.025116025116025117,
 'class-526': 0.0075985075985075985,
 'class-536': 0.01657718324384991,
 'class-540': 0.012618679285345952,
 'class-543': 0.016637849971183304,
 'class-561': 0.006309339642672976,
 'class-581': 0.023038189704856372,
 'class-597': 0.01202717869384536,
 'class-629': 0.006491339824673158,
 'class-635': 0.014453847787181121,
 'class-639': 0.02698152698152698,
 'class-653': 0.008933175599842266,

In [55]:
new_x,new_y= np.unique(temp,return_counts=True)

In [64]:
dict(zip(new_x,new_y))

{'class-1059': 0.0033275054988438327,
 'class-128': 0.008798150132536237,
 'class-140': 0.020754610569059837,
 'class-1625': 0.023630928881619762,
 'class-204': 0.1506401218205403,
 'class-239': 0.004624668659410073,
 'class-273': 0.005357847837121426,
 'class-326': 0.006542214201116689,
 'class-337': 0.04359596187468276,
 'class-362': 0.024758896847329535,
 'class-390': 0.031131915853589757,
 'class-394': 0.014437989961085106,
 'class-4': 0.017483503468501495,
 'class-448': 0.02966555749816705,
 'class-450': 0.011730866843381649,
 'class-460': 0.016975917883932098,
 'class-508': 0.02673284078732164,
 'class-526': 0.004004286278269697,
 'class-536': 0.013704810783373753,
 'class-540': 0.009813321301675032,
 'class-543': 0.016299137104506232,
 'class-561': 0.004963059049123005,
 'class-581': 0.018837065027353224,
 'class-597': 0.009756922903389543,
 'class-629': 0.005188652642264959,
 'class-635': 0.016073543511364276,
 'class-639': 0.024420506457616602,
 'class-653': 0.0064294174045457

In [58]:
new_y = new_y/new_y.sum()

In [59]:
new_y

array([0.003328, 0.008798, 0.020755, 0.023631, 0.15064 , 0.004625, 0.005358, 0.006542, 0.043596, 0.024759, 0.031132,
       0.014438, 0.017484, 0.029666, 0.011731, 0.016976, 0.026733, 0.004004, 0.013705, 0.009813, 0.016299, 0.004963,
       0.018837, 0.009757, 0.005189, 0.016074, 0.024421, 0.006429, 0.010434, 0.00987 , 0.037618, 0.061982, 0.012069,
       0.007614, 0.005583, 0.081101, 0.025774, 0.005189, 0.032767, 0.007614, 0.022785, 0.01252 , 0.077435, 0.007501,
       0.012464])

In [60]:
y

array([  889,   748,  1494,  1676, 11075,   676,   580,   637,  3471,  2148,  2292,  1230,  1495,  2565,   999,  1392,
        2058,   628,  1362,  1052,  1378,   530,  1905,  1004,   527,  1185,  2179,   714,  1069,   955,  3197,  4608,
        1147,   849,   593,  5513,  2136,   517,  2549,   908,  1771,  1131,  5517,   600,  1468], dtype=int64)

In [71]:
y = y/y.sum()

In [72]:
y

array([0.010981, 0.009146, 0.018155, 0.020035, 0.133922, 0.008145, 0.006916, 0.007811, 0.041906, 0.026527, 0.027588,
       0.014727, 0.018079, 0.030713, 0.012361, 0.016881, 0.025116, 0.007599, 0.016577, 0.012619, 0.016638, 0.006309,
       0.023038, 0.012027, 0.006491, 0.014454, 0.026982, 0.008933, 0.013134, 0.011724, 0.039327, 0.055616, 0.013787,
       0.010647, 0.007174, 0.066385, 0.025981, 0.006416, 0.030531, 0.011148, 0.021476, 0.01362 , 0.06737 , 0.007326,
       0.017669])

In [75]:
data.train_ds.y.items

array([ 0,  0,  0,  0, ..., 44, 44, 44, 44], dtype=int64)